In [2]:
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.power import TTestIndPower
from statsmodels.stats.power import FTestAnovaPower
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import dill
import pingouin as pg

## Pickle

In [3]:
NUM_STDS = 2
append_dict = {0:'reaction_time_no_sd_off',1:'reaction_time_minus_sd',2:'reaction_time_minus_2sd'}
load_path = r'D:\Subject_Data\Seth_MatchPennies_Agent_Exp1\Subjects_Analyzed\\'
subject = dill.load((open(load_path + f'subject_object_{append_dict[NUM_STDS]}_dict.pkl', 'rb')))
group   = dill.load((open(load_path + f'group_object_{append_dict[NUM_STDS]}.pkl', 'rb')))
group.analyze_data()
num_subjects = len(subject.keys())

(11, 6)


c:\Users\Seth Sullivan\OneDrive - University of Delaware - o365\Desktop\MatchPennies-Agent-Expirement\Exp1\Scripts\Analysis_and_Statistics\Subject_Object.py:562: RuntimeWarning: Mean of empty slice
  self.perc_binned_player_wins_mean                      = np.nanmean(self.combine_all_subjects('perc_binned_player_wins'),axis = 0)
c:\Users\Seth Sullivan\OneDrive - University of Delaware - o365\Desktop\MatchPennies-Agent-Expirement\Exp1\Scripts\Analysis_and_Statistics\Subject_Object.py:563: RuntimeWarning: Mean of empty slice
  self.perc_binned_player_indecisions_mean               = np.nanmean(self.combine_all_subjects('perc_binned_player_indecisions'),axis = 0)
c:\Users\Seth Sullivan\OneDrive - University of Delaware - o365\Desktop\MatchPennies-Agent-Expirement\Exp1\Scripts\Analysis_and_Statistics\Subject_Object.py:564: RuntimeWarning: Mean of empty slice
  self.perc_binned_player_incorrects_mean                = np.nanmean(self.combine_all_subjects('perc_binned_player_incorrects'),axis

## Power Analysis

In [4]:
analysis = TTestIndPower()
sample_size = analysis.solve_power(effect_size=0.5,power=0.8,alpha=0.05)


In [5]:
analysis = FTestAnovaPower()
sample_size = analysis.solve_power(effect_size=0.5,nobs=None,alpha=0.05,power=0.8,k_groups=6)
sample_size

57.10661888657371

# Create DataFrame for Pingouin

In [12]:
np.array([subject_number,condition,decision_times,wins,indecisions,incorrects]).shape

(6, 120)

In [18]:
wins = group.combine_all_subjects('player_perc_wins').flatten().tolist()
indecisions = group.combine_all_subjects('player_perc_indecisions').flatten().tolist()
incorrects = group.combine_all_subjects('player_perc_incorrects').flatten().tolist()
decision_times = group.combine_all_subjects('player_task_decision_time_mean').flatten().tolist()
subject_number = np.repeat(np.arange(0,20,1,dtype=int),6).tolist()
condition = np.tile(['np.arange(0,6,1,dtype=int)'],num_subjects).tolist()
df = pd.DataFrame(np.array([subject_number,condition,decision_times,wins,indecisions,incorrects]).T,columns=['Subject','Condition','Mean Decision Time','Wins','Indecisions','Incorrects'])
df

C:\Users\Seth Sullivan\AppData\Local\Temp\ipykernel_29836\2107752974.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df = pd.DataFrame(np.array([subject_number,condition,decision_times,wins,indecisions,incorrects]).T,columns=['Subject','Condition','Mean Decision Time','Wins','Indecisions','Incorrects'])


ValueError: Shape of passed values is (6, 1), indices imply (6, 6)

## RM Anova

Is the median decision time different between the 3 conditions (mean)?

$H_{0}$: true median decision times are the same for all 3 conditions

$H_{1}$: true median decision times are no the same for all 3 conditions

Task Decision Time Mean

In [ ]:
pg.read_dataset('mixed_anova')

,Scores,Time,Group,Subject
0,5.971435,August,Control,0
1,4.309024,August,Control,1
2,6.932707,August,Control,2
3,5.187348,August,Control,3
4,4.779411,August,Control,4
...,...,...,...,...
175,6.176981,June,Meditation,55
176,8.523692,June,Meditation,56
177,6.522273,June,Meditation,57
178,4.990568,June,Meditation,58


In [ ]:
decision_time_medians = group.combine_all_subjects('player_task_decision_time_median')
anova =pg.rm_anova((decision_time_medians[:,0],decision_time_medians[:,2],decision_time_medians[:,4]))
anova

TypeError: __init__() missing 2 required positional arguments: 'depvar' and 'subject'

Player Minus Agent Task Decision Time Mean

In [ ]:
player_minus_agent_decision_time_medians = group.combine_all_subjects('player_minus_agent_task_decision_time_mean')
anova = stats.f_oneway(decision_time_medians[:,0],decision_time_medians[:,1],decision_time_medians[:,2],decision_time_medians[:,3],decision_time_medians[:,4],decision_time_medians[:,5])
anova

F_onewayResult(statistic=10.103770704923967, pvalue=3.255048287040231e-07)

Indecisions

In [ ]:
indecisions = group.combine_all_subjects('player_perc_indecisions')
anova = stats.f_oneway(indecisions[:,0],indecisions[:,1],indecisions[:,2],indecisions[:,3],indecisions[:,4],indecisions[:,5])
anova

F_onewayResult(statistic=3.2255006541209634, pvalue=0.011495504177823114)